<a href="https://colab.research.google.com/github/chaerui7967/stock_predict_news_and_youtube/blob/master/youtube_%EC%A0%84%EC%B2%98%EB%A6%AC_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy 

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 3.6 MB/s 
     |████████████████████████████████| 448 kB 29.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


## youtube 전처리

In [3]:
import pandas as pd
import numpy as np
import datetime

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def set_data(df,com_num = 0):  # panda, numpy, datetime, FinanceDataReader, konlpy, Counter
    # 일, 시간 나누기
    df['date'] = df['date'].astype(str)
    df['date_d'] = df['date'].str[:-2]
    df['date_h'] = df['date'].str[-2:]
    # 타입을 데이트 타입으로 만듬
    df['date_d'] = pd.to_datetime(df['date_d'])
    
    df = df.sort_values(by='date_d') # 일기준으로 오름차순 정렬
    
    # 널값 제거
    df = df.dropna()
    
    # 주말 및 공휴일 데이터
    ### Holidays
    try:
        # 서버가 열려있을 때
        db = pymysql.connect(user='root',
                             passwd='1234',
                             host='3.35.70.166',
                             db='proj',
                             charset='utf8')

        cursor = db.cursor(pymysql.cursors.DictCursor)

        sql = "select * from holidays"
        cursor.execute(sql)
        result = cursor.fetchall()
    
        # DataFrame으로 변경
        holi = pd.DataFrame(result)
        # db 닫기 --> 안하면 메모리 잡아먹음
        db.close()
    except:
        # 서버 없을 때 깃허브에서 바로 가져옴
        db_holi = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/holidays.csv'
        holi = pd.read_csv(db_holi)
    
    # date 컬럼을 날짜 형식으로 변경
    holi['date'] = pd.to_datetime(holi['date'])
    
    
    ### ===================주말 및 공휴일 제외
    
    ## 주말 및 공휴일만 추출
    market_closed = holi[holi['holiday']=="O"].reset_index(drop=True)
    
    ## 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    # 주말 및 공휴일 제외
    while len(df[df['date_d'].isin(market_closed_list)]['date_d']) !=0:
        for date in df[df['date_d'].isin(market_closed_list)]['date_d'].index:
            df['date_d'][date] += datetime.timedelta(1)
    
    return df

In [6]:
youtube_df = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/youtube_total.csv')

In [7]:
youtube_df = set_data(youtube_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
youtube_df.drop(['date','length', 'url', 'date_h','description'], 1, inplace=True)

In [9]:
youtube_df = youtube_df[['st_n', 'st_cd', 'ch_nm', 'date_d', 'title', 'text', 'views']]

In [10]:
youtube_df.columns = ['st_n', 'st_cd', 'ch_nm', 'date', 'title', 'text', 'views']

In [11]:
youtube_df.head()

,st_n,st_cd,ch_nm,date,title,text,views
1362,삼성전자,5930,한국경제TV,2012-06-26,[마감시황] 삼성전자 `흔들` 코스피도 `휘청`‥1820선 하락마감,매우 늘 증시 마감 상황 정리 해보겠습니다 증권 팀 지수 이 기자나와있습니다네 오늘...,30
60,하이닉스,660,한국경제TV,2012-07-02,"SK하이닉스, M12라인 가동 차세대 메모리 본격 강화",sk 하이닉스가 차세대 메모리 반도체 사업 확장에 본격적으로 돌입했습니다청주에 새로...,154
2016,LG화학,51910,한국경제TV,2012-07-14,6인의 마감전략 성공투자 오후증시2012년 07월 13일 방송,tweet4 7월 13일 금 일 장 마감 42분 전 이군요 자 6명의 전문가는어떤 ...,115
1283,삼성전자,5930,한국경제TV,2012-07-30,삼성전자 또 사상최대 실적 2분기 영업이익 6조7천억원,소식입니다 삼성전자가 또다시 사상 최대 실적을 경신했습니다 갤럭시 시리즈의 위력을 ...,36
1259,삼성전자,5930,한국경제TV,2012-09-13,"삼성전자, 반도체 중국생산 시대 연다",삼성전자가 중국 산시성 시안 시에서 낸드플래시 반도체 공장 기공식을열었습니다삼성전자...,164


In [12]:
youtube_df.to_csv('./youtube_preprocessing.csv', index=False)

## youtube 감성분석

In [13]:
youtube_df = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/youtube_preprocessing.csv')

In [14]:
aa = youtube_df.text.iloc[:].values

## 감성분석 연습

In [15]:
import os

def read_data(filename):
    with open(filename, 'r', encoding="cp949") as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

data = read_data(r'/content/drive/My Drive/Colab Notebooks/datasets/ratings_morphed.txt')
data_text = [line[1] for line in data]    
data_senti = [line[2] for line in data]

In [16]:
from sklearn.model_selection import train_test_split

train_data_text, test_data_text, train_data_senti, test_data_senti \
    = train_test_split(
        data_text,
        data_senti,
        stratify=data_senti,
        test_size=0.3,
        random_state=156
)

In [17]:
from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
print('train_data_senti_freq:', train_data_senti_freq)
test_data_senti_freq = Counter(test_data_senti)
print('test_data_senti_freq:', test_data_senti_freq)

train_data_senti_freq: Counter({'1': 69158, '0': 69120})
test_data_senti_freq: Counter({'1': 29639, '0': 29624})


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5).fit(train_data_text)
X_train = vect.transform(train_data_text)

feature_names = vect.get_feature_names()
print("특성 개수:", len(feature_names))
print("처음 20개 특성:\m", feature_names[:20])

특성 개수: 11267
처음 20개 특성:\m ['10점', '1빠', 'cgv', 'ebs', 'kbs', 'la', 'mb', 'mbc', 'naver', 'ok', 'sbs', 'sns', 'tv', 'usb', 'ㄴㄴ', 'ㄷㄷ', 'ㅂㅅ', 'ㅅㅂ', 'ㅇㅇ', 'ㅇㅇㅇ']


In [19]:
# # Tfidf로 벡터화
# from sklearn.feature_extraction.text import TfidfVectorizer
# vect = TfidfVectorizer(min_df=5, ngram_range=(1,2)).fit(train_data_text)
# X_train = vect.transform(train_data_text)

In [20]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
y_train = pd.Series(train_data_senti)
scores = cross_val_score(LogisticRegression(solver="liblinear"), X_train, y_train, cv=5)
print('교차 검증 점수:', scores)
print('교차 검증 점수 평균:', scores.mean())

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.01, 0.1, 1, 3, 5]}
grid = GridSearchCV(LogisticRegression(solver="liblinear"), param_grid, cv=5)
grid.fit(X_train, y_train)
print("최고 교차 검증 점수:", round(grid.best_score_, 3))
print("최적의 매개변수:", grid.best_params_)

교차 검증 점수: [0.82202777 0.81667631 0.81913509 0.81518713 0.81612728]
교차 검증 점수 평균: 0.8178307153197114
최고 교차 검증 점수: 0.818
최적의 매개변수: {'C': 1}


In [21]:
X_test = vect.transform(test_data_text)
y_test = pd.Series(test_data_senti)
print("테스트 데이터 점수:", grid.score(X_test, y_test))

테스트 데이터 점수: 0.8206131987918263


In [22]:
predic = vect.transform(aa)
pred = grid.predict(predic)

### 감성사전 rhinoMorph 사용__ 그냥 다긍정으로 판단

In [22]:
!pip install rhinoMorph
!pip install JPype1

     |████████████████████████████████| 3.0 MB 12.8 MB/s 


In [23]:
import rhinoMorph
rn = rhinoMorph.startRhino()
print('rn\n',rn)
new_input = '재미 더럽 없'

# 입력 데이터 형태소 분석하기
inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
morphed_input = ' '.join(morphed_input)                     # 한 개의 문자열로 만들기
inputdata.append(morphed_input)                               # 분석 결과를 리스트로 만들기
X_input = vect.transform(inputdata)
print(float(grid.predict(X_input)))
result = grid.predict(X_input) # 0은 부정,1은 긍정
print(result)
print(type(result))
if result == 0.0:
    print("부정적인 글입니다")
else:
    print("긍정적인 글입니다")

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
RHINO started!
rn
 <java class 'rhino.RHINO'>
0.0
['0']
<class 'numpy.ndarray'>
긍정적인 글입니다


### 레이블링_ 긍정비율이 높음

In [23]:
labling = youtube_df[['date','text']]

In [24]:
labling['label'] = pred

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
labling.label.value_counts()

1    5816
0     547
Name: label, dtype: int64

## 네이버 리뷰 자료로 감성 분류

In [26]:
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [27]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f74e084b750>)

In [28]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [29]:
train_data.drop_duplicates(subset=['document'], inplace=True)

In [30]:
print('총 샘플의 수 :',len(train_data))

총 샘플의 수 : 146183


In [31]:
print(train_data.groupby('label').size().reset_index(name = 'count'))

   label  count
0      0  73342
1      1  72841


In [32]:
train_data = train_data.dropna(how = 'any')

In [33]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
train_data[:5]

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [34]:
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)

In [35]:
train_data = train_data.dropna(how = 'any')

In [36]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거

In [ ]:
okt = Okt()
url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
stopwords = list(pd.read_csv(url, header=None)[0])

In [38]:
X_train = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [39]:
X_test = []
for sentence in test_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

In [40]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [41]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 43503
등장 빈도가 2번 이하인 희귀 단어의 수: 24308
단어 집합에서 희귀 단어의 비율: 55.876606211065905
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 2.093528444535327


In [42]:
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 19196


In [43]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [44]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [45]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [46]:
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [47]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [48]:
max_len = 30
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 30 이하인 샘플의 비율: 96.11276230406929


In [49]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [50]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [51]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [52]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [53]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
1934/1934 [==============================] - 38s 14ms/step - loss: 0.3928 - acc: 0.8204 - val_loss: 0.3565 - val_acc: 0.8421

Epoch 00001: val_acc improved from -inf to 0.84205, saving model to best_model.h5
Epoch 2/15
1934/1934 [==============================] - 27s 14ms/step - loss: 0.3314 - acc: 0.8562 - val_loss: 0.3392 - val_acc: 0.8537

Epoch 00002: val_acc improved from 0.84205 to 0.85374, saving model to best_model.h5
Epoch 3/15
1934/1934 [==============================] - 27s 14ms/step - loss: 0.3067 - acc: 0.8692 - val_loss: 0.3346 - val_acc: 0.8552

Epoch 00003: val_acc improved from 0.85374 to 0.85519, saving model to best_model.h5
Epoch 4/15
1934/1934 [==============================] - 27s 14ms/step - loss: 0.2878 - acc: 0.8793 - val_loss: 0.3330 - val_acc: 0.8591

Epoch 00004: val_acc improved from 0.85519 to 0.85908, saving model to best_model.h5
Epoch 5/15
1934/1934 [==============================] - 26s 13ms/step - loss: 0.2715 - acc: 0.8883 - val_loss: 0.33

In [54]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1527/1527 [==============================] - 8s 5ms/step - loss: 0.3420 - acc: 0.8537

 테스트 정확도: 0.8537


In [69]:
def sentiment_predict(new_sentence):
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측

    if(score > 0.5):
        # print("{:.2f}% 확률로 긍정 리뷰\n".format(score * 100))
        return score, 1
    elif (score == 0.5):
        return score, 0
    else:
        # print("{:.2f}% 확률로 부정 리뷰\n".format((1 - score) * 100))
        return (1 - score), -1

In [58]:
aa = youtube_df.text.iloc[:].values

In [ ]:
scores = []
results = []
for i in aa:
    score, result = sentiment_predict(i)
    scores.append(score)
    results.append(result)

In [66]:
np.array(result.reshape(-1,1))

(0.7549134492874146,
 '매우 늘 증시 마감 상황 정리 해보겠습니다 증권 팀 지수 이 기자나와있습니다네 오늘 대장주 인 삼성전자가 국내에서 갤럭시 s3 출시 에도 불구하고큰 폭으로 하락하면서 우리 시장도 크게 흔들린 하루 였는데 자세한 내용전해주시죠네 오늘 코스피는 삼성전자의 하락 뿐이 그대로 반영된 하루였습니다시총 15% 를 차지하고 있는 삼성전자가 4% 넘게 빠지면서 시장에는 큰충격으로 다가왔습니다 오늘 코스피는 지난 금요일 보다 22.0 1p1.19% 내린 1820 5.3 팔로 마감했습니다외국인이 팔아치운 5000억원 가운데 삼성전자에서 만 3천 400억 원이빠져나갔습니다개인의 5천 5백억 원을 사들였고 기관도 1400억 원 매수 우위를보였습니다외국인 선물 매도 세로 프로그램이 2300억원 매도 우위로 마감의 지수를끌어 내렸습니다업종별로는 삼성전자 여파로 전기전자 업종이 3% 넘게 약세를 보였고 전기가서 화학 등은 소폭 상승 마감했습니다으 우빈이 매물을 쏟아내고 있는 이후 특혜의 삼성전자의 집중 매도 가나오는 이유아무래도 유럽 문제겠죠 네 그렇습니다 가장 큰 원인 유럽 2기 불확실성이투자심리를 위축시키고 있기 때문입니다최근 신흥시장 주시고 우선적으로 팔아 치우고 있는 외국인들이 국내에서가장 많이 보유하고 있는 삼성전자의 종목을 팔아 채우면서 집중 외도 되고있는 상황입니다여기에 삼성전자의 2분기 실적 전당이 기대에 미치지 못할 것이라는 신랑감도 매도세를 부추 겟습니다4 코스피 하락세 와는 달리 이제 코스닥 하락 폭은 크지 않았는데 그이유도 좀 설명해 주시죠 네 상승세로 출발한 코스닥은 낙 판 뒤 쉼부족으로 6거래일 만에 하락했습니다 오늘 코스닥은 지난 금요일 보다0.75 포인트 0.15% 내린 4 184.4 사회 장을 마감했습니다시총 상위종목 가운데 셀트리온의 약세를 보였지만 다음과 서울반도체 cj오쇼핑 등이 상승하는 등 중소형주는 선방 하는 모습을 보였습니다')